In [2]:
install.packages("shinyWidgets")

also installing the dependencies ‘BH’, ‘anytime’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [4]:
install.packages("dashboardthemes")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [21]:
install.packages("shinyBS")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [ ]:
library(shiny)
library(ggplot2)
library(stats)
library(shinyWidgets)
library(shinydashboard)
library(dashboardthemes)
library(fresh)
library(shinyBS)
library(pwr)

In [23]:
my_theme = create_theme(
  adminlte_color(
    light_blue = "#a61456"
  )
)

In [ ]:
# Function to calculate required sample size
calculate_sample_size <- function(effect_size, alpha, power) {
  pwr.t.test(power = as.numeric(power), d = effect_size, sig.level = as.numeric(alpha), alternative = "two.sided")$n
}

In [18]:
header <- dashboardHeader(
  title = "Exploration of paramaters in power calculation",
  titleWidth = 350,
  tags$li(class = "dropdown",
          tags$a(href = "#", class = "dropdown-toggle", "Logo Image", 
                 style = "padding-top: 8px;",
                 tags$img(src = "recon_logo.png", height = "30px")),
          class = "dropdown notifications-menu",
          style = "padding-right: 20px;"
  )
)

In [19]:
sidebar <- dashboardSidebar(
  sidebarMenu(
    menuItem("Basic two-sample t-test", tabName = "power_analysis")
  )
)

In [24]:
body <- dashboardBody(
  use_theme(my_theme),
  tabItems(
    tabItem("power_analysis",
            sidebarLayout(
              sidebarPanel(
                bsTooltip("effect_size", "Select an effect size. The effect size is a measure of the strength of difference between groups or conditions. A larger effect size indicates a larger difference.",
                          placement = "bottom", trigger = "hover"),
                sliderInput("effect_size", "Effect Size", min = 0.005, max = 1, value = 0.3, step = 0.005),
                bsTooltip("alpha", "The significance level is the probability of rejecting the null hypothesis when it is true.",
                          placement = "bottom", trigger = "hover"),
                selectInput("alpha", "Select significance Level", choices = c(0.01, 0.025, 0.05, 0.1), selected = 0.05),
                bsTooltip("power", "Power is the probability of correctly rejecting the null hypothesis when it is false. A higher power value indicates a higher chance of detecting a true effect if it exists.",
                          placement = "bottom", trigger = "hover"),
                selectInput("power", "Select power of test", choices = c(0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9), selected = 0.8),
                bsTooltip("name", "Select your name from dropdown for identification purposes.",
                          placement = "bottom", trigger = "hover")
              ),
              mainPanel(
                plotOutput("sample_size_plot"),
                uiOutput("sample_size_text")  
                
              )
            )
    )
  )
)

In [25]:
ui <- dashboardPage(
  header, 
  sidebar, 
  body
)

In [38]:
server <- function(input, output, session) {

      es = as.numeric(input$effect_size)
      a = as.numeric(input$alpha)
      b = as.numeric(input$power)

  
  output$sample_size_plot <- renderPlot({
    effect_sizes <- seq(0.005, 2, 0.005)
    sample_sizes <- sapply(effect_sizes, function(es) calculate_sample_size(es, a, b))
    
    df <- data.frame(Effect_Size = effect_sizes, Sample_Size = sample_sizes)
    
    ggplot(df, aes(x = Sample_Size, y = Effect_Size)) +
      geom_line(color = "#a61456") +
      geom_point(data = df[df$Effect_Size == es, ], aes(x = Sample_Size, y = Effect_Size), color = "midnightblue") +
      geom_segment(aes(y = -Inf, 
                       yend = df[df$Effect_Size == es, "Effect_Size"], 
                       x = df[df$Effect_Size == es, "Sample_Size"], 
                       xend = df[df$Effect_Size == es, "Sample_Size"]),
                   linetype = "dashed", color = "slateblue") +
      geom_segment(aes(x = -Inf, 
                       y = df[df$Effect_Size == es, "Effect_Size"], 
                       xend = df[df$Effect_Size == es, "Sample_Size"], 
                       yend = df[df$Effect_Size == es, "Effect_Size"]),
                   linetype = "dashed", color = "slateblue") +
      xlim(0, 500) +
      xlab("Sample Size") +
      ylab("Effect Size") +
      ggtitle(paste("Sample size vs. effect size for significance", a, "and power", b)) +
      theme_light()
  })
  
  
  output$sample_size_text <- renderUI({
    valueBox("Optimal sample size n (for each group)",
             width = 12,
             value = round(calculate_sample_size(effect_size= es, alpha= a, power = b)),
             color = "light-blue"
    )
  })
}

In [ ]:
# Run the Shiny app
shinyApp(ui = ui, server = server)


Listening on http://127.0.0.1:5479

